In [1]:
import numpy as np
import cv2

In [2]:
import insightface

### 1.Configure Insight Face Model

In [3]:
from insightface.app import FaceAnalysis

In [4]:
# buffalo_l model
app_l = FaceAnalysis(name='buffalo_l', 
                     root='insightface_model',
                     providers=['CPUExecutionProvider'])
app_l.prepare(ctx_id=0, det_size=(640, 640))                       

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [5]:
# load buffalo_sc model
app_sc = FaceAnalysis(name='buffalo_sc',
                     root='insightface_model',
                     providers=['CPUExecutionProvider'])
app_sc.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### 2. Load Image

In [6]:
img = cv2.imread('test_image_2.jpg')

cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [7]:
results_l = app_l.get(img)

/Volumes/STARLAI/Developments/PyCharmProject/FastFaceRegonition/venv/lib/python3.9/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [8]:
print(results_l)

[{'bbox': array([1020.8796,  182.7989, 1091.808 ,  277.7735], dtype=float32), 'kps': array([[1041.7598 ,  216.64877],
       [1075.6534 ,  216.33203],
       [1060.4402 ,  238.18265],
       [1041.0541 ,  247.29329],
       [1076.7659 ,  247.15   ]], dtype=float32), 'det_score': 0.92853606, 'landmark_3d_68': array([[ 1.0185062e+03,  2.1544214e+02,  4.7679684e+01],
       [ 1.0204072e+03,  2.2583640e+02,  4.7109615e+01],
       [ 1.0221746e+03,  2.3649060e+02,  4.5614635e+01],
       [ 1.0247014e+03,  2.4660239e+02,  4.2222572e+01],
       [ 1.0286586e+03,  2.5571912e+02,  3.4719635e+01],
       [ 1.0339337e+03,  2.6258936e+02,  2.8578329e+01],
       [ 1.0392942e+03,  2.6817712e+02,  2.3193796e+01],
       [ 1.0461715e+03,  2.7306839e+02,  1.6335123e+01],
       [ 1.0576421e+03,  2.7648807e+02,  1.3248579e+01],
       [ 1.0687531e+03,  2.7322852e+02,  1.8248960e+01],
       [ 1.0753364e+03,  2.6709991e+02,  2.7253820e+01],
       [ 1.0805253e+03,  2.6025067e+02,  3.4915977e+01],
      

In [9]:
type(results_l), len(results_l)

(list, 21)

In [10]:
results_l[0].keys()
results_l[0]['gender']

1

In [11]:
# draw the bounding box of each detected face
img_copy = img.copy()
gender_encode = ['Female', 'Male']
for res in results_l:
    x1, y1, x2, y2, = res['bbox'].astype(int)

    # draw rectangle
    cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 1)

    # key points
    kps = res['kps'].astype(int)
    for k1, k2 in kps:
        cv2.circle(img_copy, (k1, k2), 2, (0, 255, 0), -1)

    # detection score
    score = "score: {}%".format(int(res['det_score'] * 100))
    cv2.putText(img_copy, score, (x1, y1-10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 255))

    # age and gender
    gender = gender_encode[res['gender']]
    age = res['age']
    age_gender = f"{gender}::{age}"
    cv2.putText(img_copy, age_gender, (x1, y2+20), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 255))



cv2.imshow('bbox', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


-1

### Buffalo S - Model

In [12]:
# using buffalo_sc
results_sc = app_sc.get(img)

In [13]:
results_sc[0].keys()

dict_keys(['bbox', 'kps', 'det_score', 'embedding'])

In [14]:
# draw the bounding box of each detected face using buffalo_sc
img_copy = img.copy()
for res in results_sc:
    x1, y1, x2, y2, = res['bbox'].astype(int)

    # draw rectangle
    cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # key points
    kps = res['kps'].astype(int)
    for kp in kps:
        cv2.circle(img_copy, kp, 3, (0, 255, 255), -1)

    # detection score
    score = "score: {}%".format(int(res['det_score'] * 100))
    cv2.putText(img_copy, score, (x1-20, y1-10), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 0, 0))


cv2.imshow('bbox', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1